# Jsonfarmer

In [1]:
%cd ..
%load_ext dotenv
%dotenv
%env PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
%env HF_HUB_ENABLE_HF_TRANSFER=1
%env PYDEVD_DISABLE_FILE_VALIDATION=1
# !litgpt download meta-llama/Llama-3.2-1B
!litgpt download Qwen/Qwen2.5-0.5B-Instruct

/Users/antodima/Code/jsonfarmer
env: PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
env: HF_HUB_ENABLE_HF_TRANSFER=1
env: PYDEVD_DISABLE_FILE_VALIDATION=1
generation_config.json: 100%|██████████████████| 242/242 [00:00<00:00, 1.19MB/s]
model.safetensors: 100%|██████████████████████| 988M/988M [00:01<00:00, 551MB/s]
tokenizer.json: 7.03MB [00:00, 52.0MB/s]
tokenizer_config.json: 7.30kB [00:00, 11.1MB/s]
Converting checkpoint files to LitGPT format.
{'checkpoint_dir': PosixPath('checkpoints/Qwen/Qwen2.5-0.5B-Instruct'),
 'debug_mode': False,
 'dtype': None,
 'model_name': None}
Loading weights: model.safetensors: 100%|██████████████| 00:00<00:00, 103.23it/s
Saving converted checkpoint to checkpoints/Qwen/Qwen2.5-0.5B-Instruct


In [2]:
from pathlib import Path

import torch

import litgpt
from litgpt.data import JSON
from litgpt.lora import merge_lora_weights
from lightning.pytorch import Trainer, seed_everything
from jsonfarmer.model import Jsonfarmer

seed_everything(42, workers=True)
torch.set_float32_matmul_precision("high")
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

Seed set to 42


In [3]:
data = JSON(
    json_path=Path("jsonfarmer/dataset.json"),
    mask_prompt=True,
    prompt_style="alpaca",
    val_split_fraction=0.1,
)
data.prepare_data()
data.setup()
tokenizer = litgpt.Tokenizer(f"checkpoints/{model_name}")
data.connect(tokenizer, batch_size=4, max_seq_length=512)

trainer = Trainer(
    devices=1,
    max_epochs=10,
    deterministic=True,
    accumulate_grad_batches=3,
    log_every_n_steps=1,
    # fast_dev_run=True,
)
with trainer.init_module(empty_init=True):
    model = Jsonfarmer(
        model_name=model_name,
        num_tokens=tokenizer.vocab_size,
    )

trainer.fit(model, data)
# merge_lora_weights(model.model)
trainer.save_checkpoint("checkpoints/jsonfarmer.ckpt", weights_only=True)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/antodima/Code/jsonfarmer/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | model   | GPT     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/antodima/Code/jsonfarmer/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/antodima/Code/jsonfarmer/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [6]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 9794), started 0:01:13 ago. (Use '!kill 9794' to kill it.)